In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

In [2]:
from utils.ticker_finder import get_ticker_from_company_name
from memory.memory_agent import MemoryAgent
from agents.topic_classifier_agent import TopicClassifier
from agents.news_retrieval_agent import NewsRetrievalAgent
from agents.specialist_agent import Specialist

/home/bigburrito/NLP/languageEnvironment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
query = "How are apple's stocks doing?"

### 1. Getting market symbol and memory part

In [4]:
companies = ["apple", "microsoft", "google", "tesla", "nvidia", "intel"]
for company in companies:
    if company in query.lower():
        company_name = company
ticker = get_ticker_from_company_name(company_name)

In [5]:
memory_agent = MemoryAgent("../memory/history.json")

In [6]:
stored_queries = memory_agent.load_entries(ticker)

### 2. Getting the news

In [7]:
news_retrieval_agent = NewsRetrievalAgent()

limit = 3
days_back = 7

json_data = news_retrieval_agent.get_news_json(tickers=[ticker], limit_per_source=limit, days_back=days_back)

In [8]:
news = [article["summary"] for article in json_data['data'][ticker]]

### 3. Getting the topic

In [9]:
topic_classifier = TopicClassifier()

In [10]:
topic = topic_classifier.classify(query)
topic

'corporate_business'

### 4. Conversational Agent

In [11]:
specialist = Specialist(topic)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading model for topic: corporate_business from ../agents/specialized_agents/corporate_business-generator


/home/bigburrito/NLP/languageEnvironment/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/bigburrito/NLP/languageEnvironment/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-10-18 18:35:00.558602: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using GPU


As of now, past queries seem to confuse the model, the prompt structure must be adjusted

In [12]:
answer, prompt = specialist.respond(query=query, news_summaries=news, past_queries="")  #include SA once merged

In [13]:
answer

'On October 15, BofA Securities maintained its Buy rating and $270.00 price target on the stock.'

### 5. Evalution Loop

In [14]:
# Include evaluation implementation

In [15]:
feedback = ""
# answer, prompt = specialist.respond(query=query, news_summaries=news, past_queries=stored_queries, feedback=feedback)

### 6. Storage

As of now, past queries are used in the prompt, but not stored as subfields of the current query, since this is redundant 

In [16]:
entry = {
    "question": query,
    "articles": news,
    "SA": "",
    "feedback": feedback,
    "answer": answer
}
memory_agent.save_entry(entry, ticker)

In [18]:
print(prompt)

You are a domain-specific assistant for corporate_business.

Query: How are apple's stocks doing?

News Summaries:
Apple Inc. (NASDAQ:AAPL) is one of the AI Stocks Analysts Are Watching Closely. On October 15, BofA Securities maintained its Buy rating and $270.00 price target on the stock. The firm noted that iPhone shipping dates across Apple’s website and various carrier platforms over the past two weeks demonstrate stable delivery timeframes for iPhone 17 Pro […]
Saudi Arabia has ploughed more than $1bn (£744m) into a technology firm developing virtual reality headsets, which has never turned a profit.
Apple (AAPL) and Formula 1 announced a five-year partnership that will bring all F1 races exclusively to Apple TV in the United States beginning next year. “The partnership builds on Apple’s deepening relationship with Formula 1 following the global success of Apple Original Films’ adrenaline-fueled blockbuster F1 The Movie, the highest-grossing sports movie of all time. United by a c